# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'11-03-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'11-03-2020'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-11-04 05:25:09,33.93911,67.709953,41728,1544,34355,5829.0,Afghanistan,107.191827,3.700153
1,NaN,NaN,NaN,Albania,2020-11-04 05:25:09,41.15330,20.168300,21904,532,11473,9899.0,Albania,761.136980,2.428780
2,NaN,NaN,NaN,Algeria,2020-11-04 05:25:09,28.03390,1.659600,58979,1980,40577,16422.0,Algeria,134.498511,3.357127
3,NaN,NaN,NaN,Andorra,2020-11-04 05:25:09,42.50630,1.521800,4910,75,3627,1208.0,Andorra,6354.753122,1.527495
4,NaN,NaN,NaN,Angola,2020-11-04 05:25:09,-11.20270,17.873900,11577,291,5230,6056.0,Angola,35.224565,2.513605


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,40833,40937,41032,41145,41268,41334,41425,41501,41633,41728.0
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,19157,19445,19729,20040,20315,20634,20875,21202,21523,21904.0
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,56143,56419,56706,57026,57332,57651,57942,58272,58574,58979.0


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1514,1518,1523,1529,1532,1533,1536,1536,1541,1544.0
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,477,480,487,493,499,502,509,518,527,532.0
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1914,1922,1931,1941,1949,1956,1964,1973,1980,1980.0


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,34129,34150,34217,34237,34239,34258,34321,34326,34342,34355.0
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,10654,10705,10808,10893,11007,11097,11189,11246,11367,11473.0
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,39095,39273,39444,39635,39635,40014,40201,40395,40577,40577.0


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,2059,2074,2082,2103,2126,2141,2159,2173,2186,2197
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,6658,6694,6712,6743,6768,6888,6940,6966,6985,6995
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,1033,1033,1042,1045,1055,1056,1060,1061,1065,1074


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,31,31,31,31,31,31,31,31,31,31
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,69,69,69,69,69,71,71,71,71,74
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,9,9,9,9,9,9,9,9,9,9


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
634,1001.0,Autauga,Alabama,US,2020-11-04 05:25:09,32.539527,-86.644082,2197,31,0,2166.0,"Autauga, Alabama, US",3932.413324,1.411015
671,1075.0,Lamar,Alabama,US,2020-11-04 05:25:09,33.779950,-88.096680,533,7,0,526.0,"Lamar, Alabama, US",3860.919957,1.313321
672,1077.0,Lauderdale,Alabama,US,2020-11-04 05:25:09,34.901719,-87.656247,2798,43,0,2755.0,"Lauderdale, Alabama, US",3017.394774,1.536812


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,9382617,232620,3705130
India,8313876,123611,7656478
Brazil,5566049,160496,5060697


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,9382617,232620,3705130,5444867,2020-11-04 05:25:09,37.936303,-91.379001
India,8313876,123611,7656478,533787,2020-11-04 05:25:09,23.088275,81.806127
Brazil,5566049,160496,5060697,344856,2020-11-04 05:25:09,-12.669522,-48.480493
Russia,1661096,28611,1244012,388473,2020-11-04 05:25:09,54.546312,62.120860
France,1461391,37492,123664,1300235,2020-11-04 05:25:09,8.314863,-21.695174


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
Texas,950302,18632,0
California,945401,17745,0
Florida,816700,16890,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
Texas,950302,18632,0,931670,2020-11-04 05:25:09,31.660643,-98.653069
California,945401,17745,0,927656,2020-11-04 05:25:09,37.843962,-120.728594
Florida,816700,16890,0,799810,2020-11-04 05:25:09,28.940755,-82.700744
New York,513689,33543,0,480146,2020-11-04 05:25:09,42.544151,-75.474183
Illinois,436265,10161,0,426104,2020-11-04 05:25:09,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,311745,7097,0
Illinois,Cook,198184,5517,0
Florida,Miami-Dade,188636,3675,0
Texas,Harris,163845,2823,0
Arizona,Maricopa,161154,3632,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,311745,7097,0,304648,2020-11-04 05:25:09,34.308284,-118.228241,6037.0
Illinois,Cook,198184,5517,0,192667,2020-11-04 05:25:09,41.841448,-87.816588,17031.0
Florida,Miami-Dade,188636,3675,0,184961,2020-11-04 05:25:09,25.611236,-80.551706,12086.0
Texas,Harris,163845,2823,0,161022,2020-11-04 05:25:09,29.858649,-95.393395,48201.0
Arizona,Maricopa,161154,3632,0,157522,2020-11-04 05:25:09,33.348359,-112.491815,4013.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-01,41501.0,21202.0,58272.0,4825.0,11035.0,128.0,1173533.0,92254.0,27601.0,109881.0,...,1038054.0,3149.0,67156.0,92325.0,1180.0,54060.0,10.0,2063.0,16480.0,8374.0
2020-11-02,41633.0,21523.0,58574.0,4888.0,11228.0,128.0,1183131.0,93448.0,27610.0,114016.0,...,1057021.0,3165.0,67254.0,92705.0,1192.0,54775.0,10.0,2063.0,16543.0,8389.0
2020-11-03,41728.0,21904.0,58979.0,4910.0,11577.0,128.0,1195276.0,94776.0,27622.0,118198.0,...,1077099.0,3196.0,67553.0,93100.0,1202.0,55408.0,10.0,2063.0,16661.0,8410.0


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-01,1536.0,518.0,1973.0,75.0,286.0,3.0,31140.0,1363.0,907.0,1130.0,...,46807.0,59.0,570.0,801.0,35.0,489.0,1.0,600.0,349.0,243.0
2020-11-02,1541.0,527.0,1980.0,75.0,289.0,3.0,31623.0,1391.0,907.0,1159.0,...,46943.0,60.0,571.0,806.0,35.0,493.0,1.0,601.0,349.0,245.0
2020-11-03,1544.0,532.0,1980.0,75.0,291.0,3.0,32052.0,1413.0,907.0,1192.0,...,47340.0,61.0,574.0,810.0,35.0,501.0,1.0,601.0,349.0,246.0


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-01,34326.0,11246.0,40395.0,3475.0,4920.0,117.0,985316.0,55827.0,25352.0,69336.0,...,2841.0,2601.0,64466.0,87156.0,1063.0,46773.0,8.0,1366.0,15733.0,7927.0
2020-11-02,34342.0,11367.0,40577.0,3548.0,5172.0,117.0,998016.0,56466.0,25384.0,71691.0,...,2850.0,2658.0,64466.0,87547.0,1065.0,47169.0,8.0,1375.0,15733.0,7939.0
2020-11-03,34355.0,11473.0,40577.0,3627.0,5230.0,117.0,1009278.0,57599.0,25407.0,74292.0,...,2906.0,2727.0,64815.0,87941.0,1069.0,47744.0,8.0,1375.0,15763.0,7942.0


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,2059,2074,2082,2103,2126,2141,2159,2173,2186,2197
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,496,506,508,516,523,525,530,534,535,533
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,2294,2313,2338,2371,2405,2437,2465,2743,2765,2798
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,858,861,869,877,885,899,916,941,946,958
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,6546,6550,6571,6624,6649,6669,6702,6715,6736,6749


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-01,193985,16705,0,247473,113057,935878,109910,71207,25126,49,...,261426,936816,116510,2196,1376,182392,108315,24883,228862,13723
2020-11-02,194892,17057,0,248139,113641,940411,112147,73858,25311,49,...,264587,942303,117706,2219,1376,183418,109354,25238,232295,14167
2020-11-03,195929,17448,0,249818,114519,945401,114709,74843,25426,49,...,266357,950302,119375,2237,1385,184679,110011,25596,238066,14621


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-11-01        2173    6966    1061  878   2095     655   1024    4892   
2020-11-02        2186    6985    1065  883   2108     655   1026    4925   
2020-11-03        2197    6995    1074  890   2162     657   1029    4951   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-11-01         1392      787  ...     709    156      836      187   
2020-11-02         1397      791  ...     721    159      854      187   
2020-11-03         1428      808  ...     744    172      871      194   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-11-01            585   814   488          0      160    164  
2020-11-02            601   847   492          0      160    169  
2020-11-03            621   862   510          2      167    181  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-01,2973,83,0,5979,1958,17671,2288,4616,710,0,...,3353,18502,614,58,21,3655,2366,460,2047,87
2020-11-02,2973,84,0,5982,1985,17689,2292,4627,710,0,...,3379,18534,614,58,21,3658,2378,461,2050,97
2020-11-03,2987,84,0,6020,2003,17745,2311,4635,712,0,...,3454,18632,620,58,22,3666,2400,471,2102,93


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-11-01          31      71       9   15     25      17     41      65   
2020-11-02          31      71       9   15     25      17     41      65   
2020-11-03          31      74       9   15     25      17     41      65   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-11-01           47       15  ...       3      2        4        1   
2020-11-02           47       15  ...       3      2        5        1   
2020-11-03           47       15  ...       3      3        6        1   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-11-01              2     1     3         10        7      0  
2020-11-02              2     1     3         10        7      0  
2020-11-03              2     2     3          0        7      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-01,0.001835,0.015665,0.005695,0.014508,0.021286,0.0,0.005664,0.027179,0.000217,0.047234,...,0.022922,0.008003,0.003347,0.003391,0.000000,0.010090,0.0,0.0,0.002921,0.000837
2020-11-02,0.003181,0.015140,0.005183,0.013057,0.017490,0.0,0.008179,0.012943,0.000326,0.037632,...,0.018272,0.005081,0.001459,0.004116,0.010169,0.013226,0.0,0.0,0.003823,0.001791
2020-11-03,0.002282,0.017702,0.006914,0.004501,0.031083,0.0,0.010265,0.014211,0.000435,0.036679,...,0.018995,0.009795,0.004446,0.004261,0.008389,0.011556,0.0,0.0,0.007133,0.002503


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-01,0.000000,0.017682,0.004582,0.0,0.007042,0.0,0.004451,0.016406,0.0,0.018936,...,0.003473,0.017241,0.007067,0.003759,0.0,0.012422,0.0,0.001669,0.0,0.000000
2020-11-02,0.003255,0.017375,0.003548,0.0,0.010490,0.0,0.015511,0.020543,0.0,0.025664,...,0.002906,0.016949,0.001754,0.006242,0.0,0.008180,0.0,0.001667,0.0,0.008230
2020-11-03,0.001947,0.009488,0.000000,0.0,0.006920,0.0,0.013566,0.015816,0.0,0.028473,...,0.008457,0.016667,0.005254,0.004963,0.0,0.016227,0.0,0.000000,0.0,0.004082


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-01,0.000146,0.005094,0.004826,0.000000,0.087774,0.0,0.011681,0.009220,0.000632,0.031387,...,0.000704,0.016016,0.003596,0.004414,0.000942,0.010020,0.0,0.000000,0.003380,0.004180
2020-11-02,0.000466,0.010759,0.004506,0.021007,0.051220,0.0,0.012889,0.011446,0.001262,0.033965,...,0.003168,0.021915,0.000000,0.004486,0.001881,0.008466,0.0,0.006589,0.000000,0.001514
2020-11-03,0.000379,0.009325,0.000000,0.022266,0.011214,0.0,0.011284,0.020065,0.000906,0.036281,...,0.019649,0.025959,0.005414,0.004500,0.003756,0.012190,0.0,0.000000,0.001907,0.000378


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-01,0.008841,0.021400,NaN,0.006209,0.007728,0.004007,0.023847,0.000000,0.007014,0.0,...,0.002893,0.004886,0.016170,0.007802,0.000000,0.006634,0.007572,0.017294,0.015499,0.031960
2020-11-02,0.004676,0.021072,NaN,0.002691,0.005166,0.004844,0.020353,0.037229,0.007363,0.0,...,0.012091,0.005857,0.010265,0.010474,0.000000,0.005625,0.009592,0.014267,0.015000,0.032354
2020-11-03,0.005321,0.022923,NaN,0.006766,0.007726,0.005306,0.022845,0.013336,0.004543,0.0,...,0.006690,0.008489,0.014179,0.008112,0.006541,0.006875,0.006008,0.014185,0.024843,0.032046


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-11-01      0.006484  0.003746  0.000943  0.005727  0.010125  0.003063   
2020-11-02      0.005983  0.002728  0.003770  0.005695  0.006205  0.000000   
2020-11-03      0.005032  0.001432  0.008451  0.007928  0.025617  0.003053   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-11-01      0.001957  0.006377  0.002160  0.005109  ...  0.021614   
2020-11-02      0.001953  0.006746  0.003592  0.005083  ...  0.016925   
2020-11-03      0.002924  0.005279  0.022190  0.021492  ...  0.031900   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-11-01      0.012987  0.029557  0.033149   0.040925  0.014963  0.008264   
2020-11-02      0.019231  0.021531  0.000000   0.027350  0.040541  0.008197   
2020-11-03      0.081761  0.019906  0.037433   0.033278  0.017710  0.036585   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-11-01            NaN  0.006289  0.006135  
2020-11-02            NaN  0.000000  0.030488  
2020-11-03            inf  0.043750  0.071006  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-01,0.002022,0.012195,NaN,0.000000,0.017143,0.000680,0.001313,0.000000,0.002825,NaN,...,0.000000,0.002004,0.016556,0.0,0.000000,0.000274,0.000000,0.002179,0.007878,0.000000
2020-11-02,0.000000,0.012048,NaN,0.000502,0.013790,0.001019,0.001748,0.002383,0.000000,NaN,...,0.007754,0.001730,0.000000,0.0,0.000000,0.000821,0.005072,0.002174,0.001466,0.114943
2020-11-03,0.004709,0.000000,NaN,0.006352,0.009068,0.003166,0.008290,0.001729,0.002817,NaN,...,0.022196,0.005288,0.009772,0.0,0.047619,0.002187,0.009251,0.021692,0.025366,-0.041237


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                       \
Admin2         Autauga   Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-11-01         0.0  0.000000     0.0  0.0    0.0     0.0    0.0     0.0   
2020-11-02         0.0  0.000000     0.0  0.0    0.0     0.0    0.0     0.0   
2020-11-03         0.0  0.042254     0.0  0.0    0.0     0.0    0.0     0.0   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-11-01          0.0      0.0  ...     0.0    0.0     0.00      0.0   
2020-11-02          0.0      0.0  ...     0.0    0.0     0.25      0.0   
2020-11-03          0.0      0.0  ...     0.0    0.5     0.20      0.0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-11-01            0.0   0.0   0.0        0.0      0.0    NaN  
2020-11-02            0.0   0.0   0.0        0.0      0.0    NaN  
2020-11-03            0.0   1.0   0.0       -1.0      0.0    NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-01,0.002011,0.014538,0.005471,0.017201,0.022704,0.005032,0.007895,0.027251,0.000288,0.049943,...,0.023279,0.011414,0.003751,0.003858,0.000974,0.009670,1.020394e-41,0.000168,0.002733,0.001058
2020-11-02,0.002596,0.014839,0.005327,0.015129,0.020097,0.002516,0.008037,0.020097,0.000307,0.043787,...,0.020775,0.008248,0.002605,0.003987,0.005572,0.011448,5.101972e-42,0.000084,0.003278,0.001425
2020-11-03,0.002439,0.016270,0.006121,0.009815,0.025590,0.001258,0.009151,0.017154,0.000371,0.040233,...,0.019885,0.009021,0.003526,0.004124,0.006981,0.011502,2.550986e-42,0.000042,0.005206,0.001964


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-01,0.000905,0.014566,0.004321,0.004733,0.010462,3.135816e-61,0.006921,0.021401,0.000035,0.021648,...,0.005010,0.013820,0.005970,0.004794,3.449522e-20,0.011247,0.0,0.000838,0.000371,0.001218
2020-11-02,0.002080,0.015971,0.003935,0.002366,0.010476,1.567908e-61,0.011216,0.020972,0.000017,0.023656,...,0.003958,0.015384,0.003862,0.005518,1.724761e-20,0.009714,0.0,0.001252,0.000185,0.004724
2020-11-03,0.002013,0.012729,0.001967,0.001183,0.008698,7.839541e-62,0.012391,0.018394,0.000009,0.026064,...,0.006207,0.016026,0.004558,0.005241,8.623805e-21,0.012970,0.0,0.000626,0.000093,0.004403


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-01,0.000678,0.006798,0.005063,0.016397,0.083476,0.005315,0.012952,0.013303,0.000918,0.030592,...,0.004767,0.013732,0.004331,0.004842,0.000543,0.009806,5.978873e-44,0.000109,0.003531,0.003070
2020-11-02,0.000572,0.008779,0.004784,0.018702,0.067348,0.002658,0.012921,0.012374,0.001090,0.032279,...,0.003967,0.017823,0.002166,0.004664,0.001212,0.009136,2.989437e-44,0.003349,0.001765,0.002292
2020-11-03,0.000475,0.009052,0.002392,0.020484,0.039281,0.001329,0.012102,0.016220,0.000998,0.034280,...,0.011808,0.021891,0.003790,0.004582,0.002484,0.010663,1.494718e-44,0.001674,0.001836,0.001335


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-01,0.008563,0.023948,NaN,0.006475,0.009310,0.004296,0.024023,0.003149,0.007295,1.972982e-70,...,0.005105,0.006269,0.016394,0.008593,0.003499,0.007433,0.008048,0.018445,0.019415,0.030085
2020-11-02,0.006619,0.022510,NaN,0.004583,0.007238,0.004570,0.022188,0.020189,0.007329,9.864910e-71,...,0.008598,0.006063,0.013330,0.009533,0.001749,0.006529,0.008820,0.016356,0.017208,0.031219
2020-11-03,0.005970,0.022716,NaN,0.005675,0.007482,0.004938,0.022516,0.016763,0.005936,4.932455e-71,...,0.007644,0.007276,0.013755,0.008822,0.004145,0.006702,0.007414,0.015270,0.021026,0.031633


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-11-01      0.007387  0.006482  0.002424  0.006445  0.012464  0.002457   
2020-11-02      0.006685  0.004605  0.003097  0.006070  0.009335  0.001229   
2020-11-03      0.005858  0.003018  0.005774  0.006999  0.017476  0.002141   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-11-01      0.001958  0.008747  0.003343  0.008724  ...  0.021690   
2020-11-02      0.001956  0.007746  0.003468  0.006903  ...  0.019308   
2020-11-03      0.002440  0.006513  0.012829  0.014198  ...  0.025604   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-11-01      0.028443  0.028863  0.025248   0.035884  0.013366  0.011717   
2020-11-02      0.023837  0.025197  0.012624   0.031617  0.026953  0.009957   
2020-11-03      0.052799  0.022552  0.025029   0.032448  0.022331  0.023271   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-11-01      -0.999999  0.009848  0.018919  
2020-11-02      -0.999999  0.004924  0.024703  
2020-11-03      -0.999999  0.024337  0.047855  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-01,0.005195,0.021869,NaN,0.002497,0.013554,0.001666,0.002790,0.000362,0.005829,NaN,...,0.005024,0.003517,0.011941,5.713805e-29,1.907364e-07,0.001736,0.000943,0.008277,0.016089,0.009216
2020-11-02,0.002597,0.016959,NaN,0.001499,0.013672,0.001343,0.002269,0.001373,0.002915,NaN,...,0.006389,0.002623,0.005970,2.856902e-29,9.536820e-08,0.001278,0.003007,0.005225,0.008777,0.062079
2020-11-03,0.003653,0.008479,NaN,0.003926,0.011370,0.002254,0.005279,0.001551,0.002866,NaN,...,0.014293,0.003956,0.007871,1.428451e-29,2.380957e-02,0.001733,0.006129,0.013459,0.017071,0.010421


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                              \
Admin2           Autauga   Baldwin       Barbour      Bibb    Blount   
2020-11-01      0.000078  0.003627  8.514949e-09  0.001126  0.000016   
2020-11-02      0.000039  0.001813  4.257475e-09  0.000563  0.000008   
2020-11-03      0.000019  0.022034  2.128737e-09  0.000281  0.000004   

Province_State                                                        ...  \
Admin2               Bullock    Butler   Calhoun  Chambers  Cherokee  ...   
2020-11-01      2.389100e-07  0.000391  0.010490  0.001111  0.008929  ...   
2020-11-02      1.194550e-07  0.000195  0.005245  0.000555  0.004464  ...   
2020-11-03      5.972750e-08  0.000098  0.002622  0.000278  0.002232  ...   

Province_State   Wyoming                                                    \
Admin2              Park    Platte      Sheridan Sublette Sweetwater Teton   
2020-11-01      0.000061  0.007812  2.131628e-14      0.0        0.0   0.0   
2020-11-02      0.000031  0.003906  1.250000e-01      0.0        0.0   0.0   
2020-11-03      0.000015  0.251953  1.625000e-01      0.0        0.0   0.5   

Province_State                                            
Admin2             Uinta Unassigned      Washakie Weston  
2020-11-01      0.000488  -0.001115  1.589457e-07    NaN  
2020-11-02      0.000244  -0.000521  7.947286e-08    NaN  
2020-11-03      0.000122  -0.516251  3.973643e-08    NaN  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201103,AK,17448,NaN,607307,NaN,totalTestsViral,624755,97.0,NaN,...,624755,0,0,e4b7b20d6a7724e14ce53e37e1d543c6bf1176c9,0,0,0,0,0,NaN
1,20201103,AL,195929,29302.0,1196422,NaN,totalTestsViral,1363049,1022.0,20901.0,...,1392351,14,451,f6dd94c1b534c3e4a74e78d1859fcc12ae1ef103,0,0,0,0,0,NaN
2,20201103,AR,114519,9296.0,1263662,NaN,totalTestsViral,1368885,652.0,7194.0,...,1378181,18,84,fe89ba285d4babee99a637d1d0885bf35c5792be,0,0,0,0,0,NaN
3,20201103,AS,0,NaN,1768,NaN,totalTestsViral,1768,NaN,NaN,...,1768,0,0,7640aa82eb3602c10117b432fba0aed492295932,0,0,0,0,0,NaN
4,20201103,AZ,249818,6036.0,1559453,NaN,totalTestsPeopleViral,1803235,956.0,21626.0,...,1809271,38,53,aff59f7b7e0e9660b8fe9ae2b60e6f888175f2e9,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201103,AK,17448.0,NaN,607307.0,NaN,totalTestsViral,624755.0,97.0,NaN,...,624755,0,0,e4b7b20d6a7724e14ce53e37e1d543c6bf1176c9,0,0,0,0,0,NaN
1,20201103,AL,195929.0,29302.0,1196422.0,NaN,totalTestsViral,1363049.0,1022.0,20901.0,...,1392351,14,451,f6dd94c1b534c3e4a74e78d1859fcc12ae1ef103,0,0,0,0,0,NaN
2,20201103,AR,114519.0,9296.0,1263662.0,NaN,totalTestsViral,1368885.0,652.0,7194.0,...,1378181,18,84,fe89ba285d4babee99a637d1d0885bf35c5792be,0,0,0,0,0,NaN
3,20201103,AS,0.0,NaN,1768.0,NaN,totalTestsViral,1768.0,NaN,NaN,...,1768,0,0,7640aa82eb3602c10117b432fba0aed492295932,0,0,0,0,0,NaN
4,20201103,AZ,249818.0,6036.0,1559453.0,NaN,totalTestsPeopleViral,1803235.0,956.0,21626.0,...,1809271,38,53,aff59f7b7e0e9660b8fe9ae2b60e6f888175f2e9,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-11-03,AK,17448,NaN,607307,NaN,totalTestsViral,624755,97.0,NaN,NaN,...,624755,0,0,e4b7b20d6a7724e14ce53e37e1d543c6bf1176c9,0,0,0,0,0,NaN
2020-11-03,AL,195929,29302.0,1196422,NaN,totalTestsViral,1363049,1022.0,20901.0,NaN,...,1392351,14,451,f6dd94c1b534c3e4a74e78d1859fcc12ae1ef103,0,0,0,0,0,NaN
2020-11-03,AR,114519,9296.0,1263662,NaN,totalTestsViral,1368885,652.0,7194.0,246.0,...,1378181,18,84,fe89ba285d4babee99a637d1d0885bf35c5792be,0,0,0,0,0,NaN
2020-11-03,AS,0,NaN,1768,NaN,totalTestsViral,1768,NaN,NaN,NaN,...,1768,0,0,7640aa82eb3602c10117b432fba0aed492295932,0,0,0,0,0,NaN
2020-11-03,AZ,249818,6036.0,1559453,NaN,totalTestsPeopleViral,1803235,956.0,21626.0,227.0,...,1809271,38,53,aff59f7b7e0e9660b8fe9ae2b60e6f888175f2e9,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-11-03,AK,17448.0,NaN,607307.0,NaN,totalTestsViral,624755.0,97.0,NaN,NaN,...,624755,0,0,e4b7b20d6a7724e14ce53e37e1d543c6bf1176c9,0,0,0,0,0,NaN
2020-11-03,AL,195929.0,29302.0,1196422.0,NaN,totalTestsViral,1363049.0,1022.0,20901.0,NaN,...,1392351,14,451,f6dd94c1b534c3e4a74e78d1859fcc12ae1ef103,0,0,0,0,0,NaN
2020-11-03,AR,114519.0,9296.0,1263662.0,NaN,totalTestsViral,1368885.0,652.0,7194.0,246.0,...,1378181,18,84,fe89ba285d4babee99a637d1d0885bf35c5792be,0,0,0,0,0,NaN
2020-11-03,AS,0.0,NaN,1768.0,NaN,totalTestsViral,1768.0,NaN,NaN,NaN,...,1768,0,0,7640aa82eb3602c10117b432fba0aed492295932,0,0,0,0,0,NaN
2020-11-03,AZ,249818.0,6036.0,1559453.0,NaN,totalTestsPeopleViral,1803235.0,956.0,21626.0,227.0,...,1809271,38,53,aff59f7b7e0e9660b8fe9ae2b60e6f888175f2e9,0,0,0,0,0,NaN
